In [14]:
import pandas as pd
import os
from io import StringIO
import sqlite3
import csv
import geopandas as gpd
import re
from tabulate import tabulate


# Step 2A : Match Combined Random Samples

HCAD Account Number with Polygons

Download Real Property from HCAD : http://hcad.org/pdata/pdata-property-downloads.html

Real Property Data from 2025 is ~890MB, data split out available in chunked txt files.

##### *To recreate original file*


``` python
def load_real_property_chunks_safe(folder_path, delimiter='\t', log_bad_lines=True):
    files = sorted([
        f for f in os.listdir(folder_path)
        if f.startswith('real_acct_2025') and f.endswith('.txt')
    ])
    
    data_rows = []
    header = None
    bad_lines = []

    for file in files:
        file_path = os.path.join(folder_path, file)
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            lines = f.readlines()
            if header is None:
                header = lines[0].strip().split(delimiter)
                expected_cols = len(header)
            for i, line in enumerate(lines[1:], start=2):  # Start from line 2 in human terms
                parts = line.strip().split(delimiter)
                if len(parts) == expected_cols:
                    data_rows.append(parts)
                else:
                    if log_bad_lines:
                        bad_lines.append((file, i, len(parts), line.strip()))
    
    # Create DataFrame
    HCAD_RealProperty = pd.DataFrame(data_rows, columns=header)

    # Optionally print or log bad lines
    if log_bad_lines and bad_lines:
        print(f"\nSkipped {len(bad_lines)} malformed rows:")
        for file, lineno, cols, preview in bad_lines[:10]:  # show only first 10
            print(f"{file}, line {lineno}: expected {expected_cols}, found {cols} columns — {preview[:100]}...")

    return HCAD_RealProperty
```

HCAD_RealProperty = load_real_property_chunks_safe("../2025_RPD")

## Real Property Database

#### Primary Table

In [2]:
# Define Real Property Data column names
columns = [
    "acct", "yr", "mailto", "mail_addr_1", "mail_addr_2", "mail_city", "mail_state", "mail_zip", "mail_country",
    "undeliverable", "str_pfx", "str_num", "str_num_sfx", "str", "str_sfx", "str_sfx_dir", "str_unit",
    "site_addr_1", "site_addr_2", "site_addr_3", "state_class", "school_dist", "map_facet", "key_map",
    "Neighborhood_Code", "Neighborhood_Grp", "Market_Area_1", "Market_Area_1_Dscr", "Market_Area_2",
    "Market_Area_2_Dscr", "econ_area", "econ_bld_class", "center_code", "yr_impr", "yr_annexed", "splt_dt",
    "dsc_cd", "nxt_bld", "bld_ar", "land_ar", "acreage", "Cap_acct", "shared_cad", "land_val", "bld_val",
    "x_features_val", "ag_val", "assessed_val", "tot_appr_val", "tot_mkt_val", "prior_land_val",
    "prior_bld_val", "prior_x_features_val", "prior_ag_val", "prior_tot_appr_val", "prior_tot_mkt_val",
    "new_construction_val", "tot_rcn_val", "value_status", "noticed", "notice_dt", "protested",
    "certified_date", "rev_dt", "rev_by", "new_own_dt", "lgl_1", "lgl_2", "lgl_3", "lgl_4", "jurs"
]

In [3]:
def load_realacct_to_sqlite(txt_file, db_file, table_name, delimiter='\t'):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()

    # Drop and create table with appropriate schema
    cur.execute(f'DROP TABLE IF EXISTS {table_name}')
    col_defs = ', '.join([f'"{col}" TEXT' for col in columns])
    cur.execute(f'CREATE TABLE {table_name} ({col_defs})')

    # Insert valid rows in chunks
    placeholders = ','.join(['?'] * len(columns))
    with open(txt_file, 'r', encoding='utf-8', errors='replace') as f:
        reader = csv.reader(f, delimiter=delimiter)
        header = next(reader)  # Skip header in file

        batch = []
        for row in reader:
            if len(row) == len(columns):
                # Clean acct: strip and zero-pad to 13 digits
                acct_index = columns.index("acct")
                row[acct_index] = row[acct_index].strip().zfill(13)

                batch.append(row)
                if len(batch) >= 10000:
                    cur.executemany(f'INSERT INTO {table_name} VALUES ({placeholders})', batch)
                    batch = []
        if batch:
            cur.executemany(f'INSERT INTO {table_name} VALUES ({placeholders})', batch)

    conn.commit()
    conn.close()
    print(f"Loaded data into table '{table_name}' in '{db_file}'.")

In [4]:
# Load Data into Sqlite
load_realacct_to_sqlite('../2025_RPD/real_acct.txt', 'real_property.db', 'HCAD_RealProperty')

Loaded data into table 'HCAD_RealProperty' in 'real_property.db'.


In [5]:
# SQL Connect
conn = sqlite3.connect('real_property.db')
cur = conn.cursor()

In [6]:
cur.execute("""
CREATE VIEW IF NOT EXISTS HCAD_RealProperty_View AS
SELECT
    acct,
    mail_city,
    state_class,
    school_dist,
    Neighborhood_Code,
    Neighborhood_Grp,
    Market_Area_1,
    Market_Area_1_Dscr,
    Market_Area_2,
    Market_Area_2_Dscr,
    econ_area,
    econ_bld_class,
    center_code,
    yr_impr,
    yr_annexed,
    splt_dt,
    dsc_cd,
    acreage,
    land_val,
    bld_val,
    x_features_val,
    ag_val,
    assessed_val,
    tot_appr_val,
    tot_mkt_val
FROM HCAD_RealProperty;
""")

# Commit
conn.commit()

In [ ]:
HCAD_RealProperty = pd.read_sql_query("SELECT * FROM HCAD_RealProperty_View", conn)

In [ ]:
conn.close()

#### Secondary Tables

In [7]:
# Define which columns to keep from land.txt
land_columns = ["acct", "num", "use_cd", "use_dscr", "inf_cd", "inf_dscr", "inf_adj", "tp", "uts", "sz_fact",
                "inf_fact", "cond", "ovr_dscr", "tot_adj", "unit_prc", "adj_unit_prc", "val", "ovr_val"]

In [8]:
# Columns to carry over (including primary key for joining)
land_keep_columns = ["acct", "num", "use_cd", "use_dscr", "inf_cd", "inf_dscr", "ovr_dscr"]

In [9]:
def load_related_land_table(txt_file, db_file, table_name, delimiter='\t'):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()

    # Drop and create the related land table with only desired fields
    col_defs = ', '.join([f'"{col}" TEXT' for col in land_keep_columns])
    cur.execute(f'DROP TABLE IF EXISTS {table_name}')
    cur.execute(f'CREATE TABLE {table_name} ({col_defs})')

    # Index mapping based on land_columns (all original columns)
    with open(txt_file, 'r', encoding='utf-8', errors='replace') as f:
        reader = csv.reader(f, delimiter=delimiter)
        header = next(reader)  # Get the actual file header
        index_map = {col: header.index(col) for col in land_keep_columns}

        batch = []
        for row in reader:
            if len(row) == len(land_columns):  # Expecting full set of original columns
                filtered_row = [row[index_map[col]] for col in land_keep_columns]

                # Clean acct: strip and zero-pad to 13 digits
                acct_index = land_keep_columns.index("acct")
                row[acct_index] = row[acct_index].strip().zfill(13)
                batch.append(filtered_row)

                if len(batch) >= 10000:
                    placeholders = ','.join(['?'] * len(land_keep_columns))
                    cur.executemany(f'INSERT INTO {table_name} VALUES ({placeholders})', batch)
                    batch = []

        if batch:
            placeholders = ','.join(['?'] * len(land_keep_columns))
            cur.executemany(f'INSERT INTO {table_name} VALUES ({placeholders})', batch)

    conn.commit()
    conn.close()

    print(f"Loaded data into table '{table_name}' in '{db_file}'.")

In [10]:
load_related_land_table(
    txt_file="../2025_RPD/land.txt",
    db_file="real_property.db",
    table_name="land_related"
)

Loaded data into table 'land_related' in 'real_property.db'.


In [11]:
cur.execute("""
CREATE VIEW IF NOT EXISTS HCAD_CombinedView AS
SELECT
    r.acct,
    r.mail_city,
    r.state_class,
    r.school_dist,
    r.Neighborhood_Code,
    r.Neighborhood_Grp,
    r.Market_Area_1,
    r.Market_Area_1_Dscr,
    r.Market_Area_2,
    r.Market_Area_2_Dscr,
    r.econ_area,
    r.econ_bld_class,
    r.center_code,
    r.yr_impr,
    r.yr_annexed,
    r.splt_dt,
    r.dsc_cd,
    r.acreage,
    r.land_val,
    r.bld_val,
    r.x_features_val,
    r.ag_val,
    r.assessed_val,
    r.tot_appr_val,
    r.tot_mkt_val,
    l.num,
    l.use_cd,
    l.use_dscr,
    l.inf_cd,
    l.inf_dscr,
    l.ovr_dscr
FROM HCAD_RealProperty AS r
LEFT JOIN land_related AS l 
            ON r.acct = l.acct
""")
conn.commit()


In [15]:
# Preview QAQC
def preview_column_stats(db_file, table_name):
    conn = sqlite3.connect(db_file)
    cur = conn.cursor()

    # Get column names
    cur.execute(f'PRAGMA table_info({table_name})')
    columns = [row[1] for row in cur.fetchall()]

    summary = []

    for col in columns:
        cur.execute(f"""
            SELECT 
                COUNT(*) AS total_rows,
                COUNT({col}) AS non_null,
                COUNT(DISTINCT {col}) AS distinct_vals
            FROM {table_name}
        """)
        result = cur.fetchone()
        summary.append({
            "column": col,
            "total_rows": result[0],
            "non_null_values": result[1],
            "missing_values": result[0] - result[1],
            "distinct_values": result[2]
        })

    conn.close()
    df_summary = pd.DataFrame(summary)
    try:
        from tabulate import tabulate
        print(tabulate(df_summary, headers='keys', tablefmt='pretty', showindex=False))
    except ImportError:
        print(df_summary)
    return df_summary

In [16]:
preview_column_stats("real_property.db", "HCAD_CombinedView")

+--------------------+------------+-----------------+----------------+-----------------+
|       column       | total_rows | non_null_values | missing_values | distinct_values |
+--------------------+------------+-----------------+----------------+-----------------+
|        acct        |  2484951   |     2484951     |       0        |     1595816     |
|     mail_city      |  2484951   |     2484951     |       0        |      4500       |
|    state_class     |  2484951   |     2484951     |       0        |       49        |
|    school_dist     |  2484951   |     2484951     |       0        |       27        |
| Neighborhood_Code  |  2484951   |     2484951     |       0        |      8568       |
|  Neighborhood_Grp  |  2484951   |     2484951     |       0        |       979       |
|   Market_Area_1    |  2484951   |     2484951     |       0        |       400       |
| Market_Area_1_Dscr |  2484951   |     2484951     |       0        |       229       |
|   Market_Area_2    

,column,total_rows,non_null_values,missing_values,distinct_values
0,acct,2484951,2484951,0,1595816
1,mail_city,2484951,2484951,0,4500
2,state_class,2484951,2484951,0,49
3,school_dist,2484951,2484951,0,27
4,Neighborhood_Code,2484951,2484951,0,8568
5,Neighborhood_Grp,2484951,2484951,0,979
6,Market_Area_1,2484951,2484951,0,400
7,Market_Area_1_Dscr,2484951,2484951,0,229
8,Market_Area_2,2484951,2484951,0,88
9,Market_Area_2_Dscr,2484951,2484951,0,88


In [17]:
HCAD_RealProperty = pd.read_sql_query("SELECT * FROM HCAD_CombinedView", conn)

In [19]:
conn.close()

In [ ]:
HCAD_RealProperty

## Analysis

In [20]:
gdf_allsamples = gpd.read_file("OUTPUT\combined_random_samples.geojson")

In [ ]:
gdf_allsamples

##### check step

In [21]:
print("HCAD_NUM samples from gdf:")
print(gdf_allsamples["HCAD_NUM"].dropna().astype(str).unique()[:5])

HCAD_NUM samples from gdf:
['1274410000036' '1152790430005' '1219740030067' '1416470010001'
 '0432120010343']


In [22]:
print("\nacct samples from HCAD_RealProperty:")
print(HCAD_RealProperty["acct"].dropna().astype(str).unique()[:5])


acct samples from HCAD_RealProperty:
['0010010000013' '0010020000001' '0010020000003' '0010020000004'
 '0010020000013']


In [23]:
gdf_accts = set(gdf_allsamples["HCAD_NUM"].dropna().astype(str))
db_accts = set(HCAD_RealProperty["acct"].dropna().astype(str))

common = gdf_accts & db_accts
print(f"Matching account count: {len(common)} of {len(gdf_accts)} in gdf")

Matching account count: 6072 of 6076 in gdf


##### Steps to Match (now archived)
This has now been moved up to the sql inmport stage, but kept to maintain the "clean" tag

In [24]:
gdf_allsamples["HCAD_NUM_clean"] = gdf_allsamples["HCAD_NUM"] #.astype(str).str.strip().str.zfill(13)
HCAD_RealProperty["acct_clean"] = HCAD_RealProperty["acct"] #.astype(str).str.strip().str.zfill(13)

In [25]:
common_clean = set(gdf_allsamples["HCAD_NUM_clean"]) & set(HCAD_RealProperty["acct_clean"])
print(f"Matches after cleaning: {len(common_clean)}")


Matches after cleaning: 6072


##### actual join
joins RealProperty Data View from db into the selected samples

In [26]:
gdf_joined = gdf_allsamples.merge(HCAD_RealProperty, 
                                  left_on="HCAD_NUM_clean", 
                                  right_on="acct_clean", 
                                  how="left")

In [27]:
columns_to_drop = ["parcel_type", 
                   "yr_annexed", 
                   "splt_dt", 
                   "dsc_cd",
                   "timestamp",
                   "sampling_batch_size",
                   "total_properties_sampled",
                   "sampling_attempts",
                   "samplesetID"
                   ]
gdf_joined = gdf_joined.drop(columns=columns_to_drop, errors="ignore")

In [28]:
front_cols = [
    'parcel_index', 
    'unique_id', # unique record ID
    'sample_id', # sample set identifier
    'HCAD_NUM', 
    'HCAD_NUM_clean', 
    'acct_clean',
    'city', 
    'mail_city', 
    'state_class_y', 
    'state_class_x',
    'use_dscr',
    'appr_val', 
    'tot_appr_val', 
    'mkt_val', 
    'tot_mkt_val'
]

In [29]:
remaining_cols = [col for col in gdf_joined.columns if col not in front_cols]

In [30]:
gdf_joined = gdf_joined[front_cols + remaining_cols]

#### Data Type Conversion

In [31]:
# Columns to convert
currency_cols = ['appr_val', 'tot_appr_val', 'mkt_val', 'tot_mkt_val']
conversion_report = {}

In [32]:
# Column Conversion
for col in currency_cols:
    if col in gdf_joined.columns:
        original_nulls = gdf_joined[col].isna().sum()
        original_type = gdf_joined[col].dtype

        # Clean and convert
        gdf_joined[col] = (
            gdf_joined[col]
            .replace('', '0')             # Replace empty strings with '0'
            .replace(' ', '0')            # Replace space-only strings too
            .fillna(0)                    # Fill any remaining NaNs
            .astype(float)                # Convert to float
            .round(2)
        )

        # Confirmation
        conversion_report[col] = {
            'original_dtype': str(original_type),
            'nulls_filled': original_nulls,
            'total_records': len(gdf_joined)
        }


In [33]:
# Print conversion report
print("Currency Conversion Summary:")
for col, stats in conversion_report.items():
    print(f" - {col}: {stats['total_records']} rows processed | {stats['nulls_filled']} nulls filled | original type was {stats['original_dtype']}")

Currency Conversion Summary:
 - appr_val: 19190 rows processed | 816 nulls filled | original type was float64
 - tot_appr_val: 19190 rows processed | 8 nulls filled | original type was object
 - mkt_val: 19190 rows processed | 816 nulls filled | original type was float64
 - tot_mkt_val: 19190 rows processed | 8 nulls filled | original type was object


#### Integrity Report

In [34]:
mismatch_report = {}

In [35]:
def check_and_cleanup(col1, col2, drop_col_if_match, rename_if_match=None):
    global gdf_joined
    total = len(gdf_joined)

    if col1 not in gdf_joined.columns or col2 not in gdf_joined.columns:
        print(f"Warning: One or both columns missing: {col1}, {col2}")
        return

    if (gdf_joined[col1] == gdf_joined[col2]).all():
        if drop_col_if_match and drop_col_if_match in gdf_joined.columns:
            gdf_joined.drop(columns=[drop_col_if_match], inplace=True)
        if rename_if_match:
            gdf_joined.rename(columns={col2: rename_if_match}, inplace=True)
    else:
        mismatch_count = (gdf_joined[col1] != gdf_joined[col2]).sum()
        mismatch_report[f"{col1} vs {col2}"] = {
            "count": mismatch_count,
            "percent": round((mismatch_count / total) * 100, 2)
        }

In [ ]:
# HCAD_NUM comparisons
check_and_cleanup("HCAD_NUM", "HCAD_NUM_clean", drop_col_if_match="HCAD_NUM_clean")
check_and_cleanup("HCAD_NUM", "acct_clean", drop_col_if_match="acct_clean")

# State class comparison
check_and_cleanup("state_class_y", "state_class_x", drop_col_if_match="state_class_x", rename_if_match="state_class")

# Appraised value
check_and_cleanup("appr_val", "tot_appr_val", drop_col_if_match="appr_val")

# Market value
check_and_cleanup("mkt_val", "tot_mkt_val", drop_col_if_match="mkt_val")

In [36]:
# Print mismatch report if any
if mismatch_report:
    print("\nColumn Mismatches Found:")
    for k, stats in mismatch_report.items():
        print(f"{k}: {stats['count']} records do not match ({stats['percent']}% of total)")
else:
    print("All compared columns are fully aligned; cleanup applied.")

All compared columns are fully aligned; cleanup applied.


In [ ]:
gdf_joined

# Step 2B : Sample Matching Analysis

In [37]:
# Save point
gdf_joined.to_file("OUTPUT/gdf_joined_output.geojson", driver="GeoJSON")

In [38]:
# From Save Point, re-load all geospatial files
metro_boundary = gpd.read_file("../REF/Metro_MTA_Tax_Area.geojson")
metro_RRC = gpd.read_file("../REF/METRO_RailRoadCrossings.geojson")
metro_BB = gpd.read_file("../REF/METRO_TxDOT_Bridges.geojson")

metro_TC = gpd.read_file("../REF/Metro_Transit_Centers.geojson")
metro_PnR = gpd.read_file("../REF/Metro_Park_and_Rides.geojson")
metro_LRT = gpd.read_file("../REF/Metro_LRT_Stations.geojson")

metro_rs_gdf = gpd.read_file("OUTPUT/gdf_joined_output.geojson")

# Clearnin up Raw Data Import
pnr_col2k = ["OBJECTID",
             "ADDRESS",
             "ROUTES_SER",
             "geometry"
            ]
metro_PnR = metro_PnR[pnr_col2k]
metro_TC = metro_TC[pnr_col2k]

lrt_col2k = ["OBJECTID",
             "Stat_Name",
             "Stat_Loc",
             "LineColor",
             "geometry"
            ]
metro_LRT = metro_LRT[lrt_col2k]

bb_col2k = ["BRDG_ID",
            "FEAT_INTSECTD",
            "FACLTY_CARRD_BY_STRUC",
            "YR_BLT",
            "ADT",
            "ADT_YR",
            "ADT_TRK",
            "LT_SDWALK_WIDTH",
            "RT_SDWALK_WIDTH",
            "FUT_ADT",
            "FUT_ADT_YR",
            "BRDG_DESC",
            "geometry"
            ]
metro_BB = metro_BB[bb_col2k]

rrc_col2k = ["CrossingID",
             "Street",
             "TtstnNam",
             "TypeXing",
             "Railroad",
             "geometry"
             ]
metro_RRC = metro_RRC[rrc_col2k]

In [39]:
def apply_overlay_analysis(parcel_gdf, features):
    # Project all layers to a suitable linear CRS (Houston = EPSG:2278)
    crs_proj = "EPSG:2278"
    parcel_gdf = parcel_gdf.to_crs(crs_proj)
    for key in features:
        features[key] = features[key].to_crs(crs_proj)

    # Distance to nearest transit access points
    parcel_gdf["dist_to_tcenter"] = parcel_gdf.geometry.apply(lambda x: features["tcenter"].distance(x).min())
    parcel_gdf["dist_to_parkride"] = parcel_gdf.geometry.apply(lambda x: features["parkride"].distance(x).min())
    parcel_gdf["dist_to_lrt"] = parcel_gdf.geometry.apply(lambda x: features["lrt"].distance(x).min())

    # Barrier analysis: 1-mile buffer (1609 meters)
    buffer_1mile = parcel_gdf.geometry.buffer(1609)
    parcel_gdf["barrier_bridges"] = [features["bridges"].intersects(b).sum() for b in buffer_1mile]
    parcel_gdf["barrier_rxings"] = [features["rrc"].intersects(b).sum() for b in buffer_1mile]

    # Accessibility analysis: ¼-mile buffer (402 meters)
    buffer_quarter = parcel_gdf.geometry.buffer(402)
    parcel_gdf["access_tcenter_qtrmi"] = [features["tcenter"].intersects(b).sum() for b in buffer_quarter]
    parcel_gdf["access_parkride_qtrmi"] = [features["parkride"].intersects(b).sum() for b in buffer_quarter]
    parcel_gdf["access_lrt_qtrmi"] = [features["lrt"].intersects(b).sum() for b in buffer_quarter]

    return parcel_gdf.to_crs("EPSG:4326")  # Restore to geographic coordinates for export


In [40]:
features_dict = {
    "tcenter": metro_TC,
    "parkride": metro_PnR,
    "lrt": metro_LRT,
    "bridges": metro_BB,
    "rrc": metro_RRC
}

metro_rs_gdf_overlay = apply_overlay_analysis(metro_rs_gdf, features_dict)

In [41]:
# Overlay Save Point result
metro_rs_gdf_overlay.to_file("OUTPUT/metro_rs_overlay.geojson", driver="GeoJSON")

In [ ]:
metro_rs_gdf_overlay